In [ ]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import pickle

print('TensorFlow version: {}'.format(tf.__version__))

In [ ]:
from db_actions import DatabaseActions as db

data = db.get_all()
print(data[3])

In [ ]:
dataset = [[data[0][4], pickle.loads(bytes.fromhex(data[0][7]))]]
for i in range(1, len(data)):

    if(data[i][1] == 5):
        dataset.append([data[i][4], pickle.loads(bytes.fromhex(data[i][7]))])


In [ ]:

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

vectors_tmp = []
for record in dataset:
    vectors_tmp.append(record[1])

# make numpy array vectors
vectors = np.array(vectors_tmp)

print(vectors.shape)


In [ ]:
# print max value in vector in vectors by absolute value
def max_abs_value(vectors):
    max_value = 0
    for vector in vectors:
        max_value = max(max_value, max(abs(x) for x in vector))
    return max_value

print(max_abs_value(vectors), end= '\n' + '~' * 80 + '\n')

vectors = vectors / max_abs_value(vectors)

print(max_abs_value(vectors))


In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=768, activation='relu'))
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(768, activation='sigmoid'))
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')
# Train the model
model.fit(vectors, np.array([[1,0]]*len(vectors)), epochs=10, verbose=1)



In [ ]:
predicted_vectors = model.predict(vectors)

# Apply K-means clustering to the predicted vectors
kmeans = KMeans(n_clusters=2)
labels = kmeans.fit_predict(predicted_vectors)

In [ ]:
# import structure.outputs
c = 0
for i in range(labels.shape[0]):
    if labels[i] == 0:
        c += 1
        print(dataset[i][0],end = '\n\n' + '-' * 160 + '\n\n')

print(c)

In [ ]:
c = 0
for i in range(labels.shape[0]):
    if labels[i] == 1:
        c += 1
        print(dataset[i][0],end = '\n\n' + '-' * 160 + '\n\n')

print(c)